# Анализ данных и проверка методов обработки данных 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from data_description import get_data_frame_settings
from processing import obj_to_cat


import DS_common.emissions as em
from DS_common.excel_writing import save_double_column_df
from sys import path

path.append(
    '/home/dranik/KFA/university/diplom_project/prog/data_processing/classification_power_predictor'
)

from classification_power_predictor\
import classification_power_predictor as cpp

# Загрузка данных

In [4]:
data = obj_to_cat(pd.read_csv(
    "/home/dranik/KFA/university/proc_1.csv"
))

# Очистка от выбросов

**Мы не удаляем строки - а прописываем выбросам пропуск**
Почему мы так делаем?

1) Что-бы не удалять больше чем нужно;<br>
2) Что-бы оставлось побольше данных - нет смысла много удалять пока не оценена классифицирующая способность.

Отрицительный стаж работы - бред!

In [5]:
data.loc[
    data['Работа последнее место стаж лет'] > 0,
    'Работа последнее место стаж лет'
] = np.NaN

Отрицительный доход - бред. Да и на 100 рублей не выжить.

In [6]:
data.loc[
    data['Работа уровень дохода BYR'] > 100,
    'Работа уровень дохода BYR'
] = np.NaN

Неположитеьная сумма договора - бред

In [7]:
data.loc[
    data['Сумма договора'] < 0,
    'Сумма договора'
] = np.NaN

Ежедневный платеж

In [8]:
data.loc[
    data['Ежедневный платеж'] < 0,
    'Ежедневный платеж'
] = np.NaN

**Показались подозлительными размахи переменных**

1.Количество фактов просрочки по основному долгу;<br>
2.Максимальное количество дней просрочки;<br>
3.Общее количество запросов в КБ;<br>
4.Сумма кредитных лимитов;<br>
5.Сумма договора;<br>
6.ежедневный платеж.

In [9]:
em_research_list = [
    "Количество фактов просрочки по основному долгу",
    "Максимальное количество дней просрочки",
    "Общее количество запросов в КБ",
    "Сумма кредитных лимитов", "Сумма договора",
    "Ежедневный платеж"
]

Рассмотрим описание данных

In [10]:
def get_describes(df):
    result = pd.DataFrame()
    for col in df:
        result[col] = df[col].describe()
    return result

get_describes(data[em_research_list]).\
to_excel("result/emissions_describes_initial.xlsx")

Ну гляза на максимальное значений и 75-е персентили все становиться понятно - проведем удаление по правилу из Википедии.

In [11]:
def cut_emissioins(col):
    return col[em.get_selcond_emiss_25_75(col, cut_type = 'right')]

data[em_research_list] = \
data[em_research_list].apply(cut_emissioins)

In [12]:
#get_describes(data[em_research_list])
get_describes(data[em_research_list]).\
to_excel("result/emissions_describes_new.xlsx")

# Классифицирующая способность

In [25]:
def cl_abby(data, file_name):
    object_data = data.copy()
    cat_cols = object_data.columns[object_data.dtypes == 'category']
    object_data[cat_cols] = object_data[cat_cols].astype("O")

    my_cpp = cpp(
        object_data.drop('Y', axis = 1), 
        object_data['Y'].replace({0:"Нет дефолта", 1:"Дефолт"}))
    my_cpp.update_predictors()
    
    writer = pd.ExcelWriter(file_name)
    save_double_column_df(my_cpp.result_DF,writer)
    writer.close()
    return my_cpp.result_DF

result_DF = cl_abby(data, "result/classify_abby/1.xlsx")

## По незначимости KS статистики выборку покидают

In [32]:
result_DF = pd.read_excel(
    "result/classify_abby/1.xlsx",
    index_col = 0, header=[0,1]
)


drop_index = result_DF[
    result_DF[("Дефолт","KS_p_val")] > 0.05
].index
drop_index.tolist()

['Количество детей',
 'Отношение к банку',
 'Работа последнее место стаж лет',
 'Работа уровень дохода BYR',
 'Количество действующих договоров обеспечения',
 'Число авто',
 'Есть авто']

In [33]:
data.drop(drop_index, inplace = True, axis = 1)

cl_abby(data, "result/classify_abby/2.xlsx")

### Показатели оказавшиеся хуже связанных

In [34]:
drop_cols = [
    'Ежедневный платеж',
    'Автомобиль год выпуска'
]

data.drop(drop_cols, inplace = True, axis = 1)

## Далее борьба с пропусками

Воинская служба много пропусков - но естесвенно понятно, что если мы укажем "невоеннообязанный" для всеж Ж у который пропуск то в 99% случаях окажемся правы.

In [35]:
data.loc[data['Пол'] == 'Ж', 'Воинская служба'] = "невоеннообязанный"

### Автомобили

Интересная ситуация с автомобилями - ниодна из моих попыток объединить не увенчалась успехом, но сами по себе они оказались вполне хорошими показателями. Можно перекодировать в номинативные.

In [39]:
auto_names = data.columns[
    data.columns.str.find("Автомобиль") == 0
]

Бинарный вариант - есть данные, нет данных

In [43]:
data[auto_names + " бинарный"] = \
data[auto_names].apply(lambda x: x.isna())

In [44]:
#data[auto_names + " тройной"] = 

,Автомобиль год выпуска1,Автомобиль год выпуска2,Автомобиль год выпуска3,Воинская служба,Количество иждивенцев,Недвижимость,Образование,Работа занимаемая должность,Семейное положение,Собственная квартира,...,Сумма договора,Количество потребляемых банковских продуктов,Пол,Социальная группа,Код подразделения,Срок кредита в днях,Y,Автомобиль год выпуска1 бинарный,Автомобиль год выпуска2 бинарный,Автомобиль год выпуска3 бинарный
0,2953.0,NaN,NaN,военнослужащий,NaN,есть,среднее специальное,специалист,женат/замужем,есть,...,400.0,более 2-х,М,служащий,739-600,351.0,0,False,True,True
1,NaN,NaN,NaN,невоеннообязанный,NaN,нет,среднее специальное,рабочий,холост/не замужем,нет,...,1000.0,более 2-х,Ж,работающий по найму,739-900,349.0,0,True,True,True
2,2955.0,NaN,NaN,уволен в запас,NaN,есть,среднее специальное,специалист,холост/не замужем,есть,...,900.0,более 2-х,М,работающий по найму,739-900-500,349.0,0,False,True,True
3,-696.0,NaN,NaN,невоеннообязанный,NaN,нет,высшее,руководитель,женат/замужем,нет,...,500.0,более 2-х,М,работающий по найму,739-400,350.0,0,False,True,True
4,NaN,NaN,NaN,невоеннообязанный,NaN,нет,среднее специальное,специалист,холост/не замужем,нет,...,750.0,более 2-х,Ж,работающий по найму,739-600,351.0,0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236492,7315.0,NaN,NaN,невоеннообязанный,2.0,нет,высшее,специалист,женат/замужем,нет,...,NaN,NaN,Ж,работающий по найму,739-100,7280.0,0,False,True,True
236493,NaN,NaN,NaN,невоеннообязанный,NaN,нет,высшее,рабочий,женат/замужем,нет,...,1500.0,более 2-х,Ж,работающий по найму,739-600,1798.0,0,True,True,True
236494,6221.0,NaN,NaN,уволен в запас,NaN,нет,высшее,рабочий,женат/замужем,нет,...,NaN,NaN,М,работающий по найму,739-100,7279.0,0,False,True,True
236495,NaN,NaN,NaN,невоеннообязанный,NaN,нет,высшее,специалист,женат/замужем,нет,...,NaN,NaN,Ж,работающий по найму,739-100,7278.0,0,True,True,True
